In [1]:
import sys
sys.path.append('/home/de-user/code/de-project-final-repo')

from src.clients.s3 import get_client

/home/de-user/code/de-project-final-repo/src/logger/__init__.py:41: RuntimeWarning: Logging handler is not specified. Default value will be used -> console. If you want to change set handler as environment variable LOGGING_HANDLER
  warnings.warn(


In [2]:
s3 = get_client()

In [3]:
import pandas as pd

pd.options.display.max_columns = None
pd.options.display.max_rows = None

In [10]:
path_to_remove = 's3a://data-ice-lake-05/dev/source/spark-streaming-checkpoints'

s3.rm(path=path_to_remove, recursive=True)

[]

In [11]:
s3.ls('s3a://data-ice-lake-05/dev/source')

['data-ice-lake-05/dev/source/transaction-service']

In [37]:
path = "s3a://data-ice-lake-05/dev/source/transaction-service/object_type=transaction/date=2023-10-07/hour=18/batch_id=0"

s3.ls(path)

['data-ice-lake-05/dev/source/transaction-service/object_type=transaction/date=2023-10-07/hour=18/batch_id=0/part-00000-4c34a093-a26c-4c5d-bdc3-214ed33f7058.c000.snappy.parquet']

In [38]:
with s3.open(s3.ls(path)[0], 'rb') as f:
    df = pd.read_parquet(f)

In [39]:
df.head()

,object_id,sent_dttm,payload,trigger_dttm
0,e2d4cfd9-4dc6-4a02-8c7a-c391a420206b,2022-10-01 00:11:45,"{""operation_id"":""e2d4cfd9-4dc6-4a02-8c7a-c391a...",2023-10-07 18:44:34.385775
1,b05f56ef-3b59-49ee-851c-a37a98d1dd28,2022-10-01 00:22:28,"{""operation_id"":""b05f56ef-3b59-49ee-851c-a37a9...",2023-10-07 18:44:34.385775
2,a62136ce-f65a-45d8-8472-18bdc5c1e0e3,2022-10-01 00:28:04,"{""operation_id"":""a62136ce-f65a-45d8-8472-18bdc...",2023-10-07 18:44:34.385775
3,a0d7ccfa-8bf1-4921-8d68-a50e5e2b2fbd,2022-10-01 00:29:53,"{""operation_id"":""a0d7ccfa-8bf1-4921-8d68-a50e5...",2023-10-07 18:44:34.385775
4,818b7011-e7e0-48c1-a0e3-8c061c92fc1f,2022-10-01 00:38:23,"{""operation_id"":""818b7011-e7e0-48c1-a0e3-8c061...",2023-10-07 18:44:34.385775
